# E2E registered components

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define `commandComponent` using YAML
- Register it to workspace
- Create `pipeline` using these registered components

**Motivations** - This notebook explains how to register via SDK then use these components to build pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
#import required libraries
from azure.ml import MLClient
from azure.ml.entities import JobInput 
from azure.identity import InteractiveBrowserCredential
from azure.ml._constants import AssetTypes
from azure.ml import dsl
from azure.ml.entities import load_component

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [interactive authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.interactivebrowsercredential?view=azure-python) for this tutorial. More advanced connection methods can be found [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(InteractiveBrowserCredential(), subscription_id, resource_group, workspace)

# 3. Pipeline job with registered component
## 3.1 Register components

In [ ]:
parent_dir = '.'
train_component = load_component(yaml_file=parent_dir + "/train.yml")
ml_client.components.create_or_update(train_component)


In [ ]:
score_component = load_component(yaml_file=parent_dir + "/score.yml")
ml_client.components.create_or_update(score_component)

In [ ]:
eval_component = load_component(yaml_file=parent_dir + "/eval.yml")
ml_client.components.create_or_update(eval_component)

## 3.2 Build pipeline

In [ ]:
# 1. Load component funcs
train_func = load_component(
    client=ml_client,
    name=train_component.name,
    version=train_component.version,
)
score_func = load_component(
    client=ml_client,
    name=score_component.name,
    version=score_component.version,
)
eval_func = load_component(
    client=ml_client,
    name=eval_component.name,
    version=eval_component.version,
)

# 2. Construct pipeline
@dsl.pipeline(
    compute="cpu-cluster",
    description="E2E dummy train-score-eval pipeline with registered components",
)
def sample_pipeline(
        pipeline_job_training_input,
        pipeline_job_test_input,
        pipeline_job_training_max_epocs,
        pipeline_job_training_learning_rate,
        pipeline_job_learning_rate_schedule,
):
    train_job = train_func(
        training_data=pipeline_job_training_input,
        max_epocs=pipeline_job_training_max_epocs,
        learning_rate=pipeline_job_training_learning_rate,
        learning_rate_schedule=pipeline_job_learning_rate_schedule,
    )
    score_job = score_func(model_input=train_job.outputs.model_output, test_data=pipeline_job_test_input)
    score_job.outputs.score_output.mode = "upload"
    evaluate_job = eval_func(scoring_result=score_job.outputs.score_output)
    return {
        "pipeline_job_trained_model": train_job.outputs.model_output,
        "pipeline_job_scored_data": score_job.outputs.score_output,
        "pipeline_job_evaluation_report": evaluate_job.outputs.eval_output,
    }

pipeline = sample_pipeline(
    JobInput(type=AssetTypes.URI_FOLDER, path=parent_dir + "/data/"),
    JobInput(type=AssetTypes.URI_FOLDER, path=parent_dir + "/data/"),
    20,
    1.8,
    "time-based",
)
pipeline.outputs.pipeline_job_trained_model.mode = "upload"
pipeline.outputs.pipeline_job_scored_data.mode = "upload"
pipeline.outputs.pipeline_job_evaluation_report.mode = "upload"

## 3.3 Submit pipeline job

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(pipeline, experiment_name="e2e_registered_components", continue_run_on_step_failure=True)
pipeline_job.services["Studio"].endpoint

# Next Steps
You can see further examples of running a pipeline job [here](/sdk/jobs/pipelines/)